In [1]:
!pip install --upgrade pandas

    100% |████████████████████████████████| 14.9MB 77kB/s eta 0:00:01
    100% |████████████████████████████████| 4.7MB 244kB/s ta 0:00:011
    100% |████████████████████████████████| 512kB 1.7MB/s ta 0:00:01
Requirement already up-to-date: python-dateutil>=2 in /Users/YK/anaconda3/lib/python3.6/site-packages (from pandas)
Requirement already up-to-date: six>=1.5 in /Users/YK/anaconda3/lib/python3.6/site-packages (from python-dateutil>=2->pandas)
  Found existing installation: numpy 1.13.3
    Uninstalling numpy-1.13.3:
      Successfully uninstalled numpy-1.13.3
  Found existing installation: pytz 2017.2
    Uninstalling pytz-2017.2:
      Successfully uninstalled pytz-2017.2
  Found existing installation: pandas 0.20.3
    Uninstalling pandas-0.20.3:
      Successfully uninstalled pandas-0.20.3


In [ ]:
!pip install gensim

In [156]:
!pip install tqdm

    100% |████████████████████████████████| 61kB 1.9MB/s ta 0:00:01


In [2]:
import os

In [3]:
import nltk
from nltk import word_tokenize
import gensim
import pandas as pd
from tqdm import tqdm

In [193]:
# constructing the tree: string to sequence of its elements
def split(tree_str):
    chunks = tree_str.split()
    chunk_elements = []
    for chunk in chunks:
        if chunk[0] == "(":
            index_i = 0
            while chunk[index_i] == "(":
                chunk_elements.append("(")
                index_i += 1
            chunk_elements.append(chunk[index_i:])
        else:
            index_j = (-1)
            while chunk[index_j] == ")":
                chunk_elements.append(")")
                index_j += (-1)
            chunk_elements.insert((index_j + 1), chunk[0:((index_j)+1)])
    return(chunk_elements)

In [207]:
# generating constituents
def constituents(tree_str):
# nt = non-terminal
    nt_seq = []
    term_counter = 0
    constituents_set = set()
    parsed_str = split(tree_str)
    element_index = 0
    none_depth = 0
    depth = 0
    while element_index < len(parsed_str):
        if parsed_str[element_index] == "(":
            nt = parsed_str[element_index + 1]
            depth += 1
            if nt == "-NONE-":
# we are entering a subtree beginning with -NONE-                
                none_depth += 1
                nt_seq.append((nt, term_counter + 1))
            else:
# taking first of the nt-label
                if len(nt) > 0 and nt[0] == "-":
                    first_nt_label = nt
                else:
                    first_nt_label = nt.split("-")[0]
                nt_seq.append((first_nt_label, term_counter + 1))
#  skipping a non-terminal           
            element_index += 2
        elif parsed_str[element_index] == ")":
            last_nt = nt_seq.pop()
            depth -= 1
            if last_nt[0] == "-NONE-":
# exiting a subtree beginning with -NONE-                
                none_depth -= 1
            else:
                if last_nt[1] <= term_counter:
                    if last_nt[0] == "ROOT" or last_nt[0] =="TOP":
                        pass
                    else:
                        constituents_set.add((last_nt[0], last_nt[1], term_counter, depth))
                else:
# In this case all terminals in the subtree were ignored, which means that the subtree contained nothing but traces, 
# therefore we're skipping the constituent.
                    pass
            element_index += 1
        else:
            if none_depth == 0:
# if we are not in a subtree beginning with -NONE-, then we count the terminal  
                term_counter += 1
#                 print(term_counter, parsed_str[element_index])
            else:
# otherwise we ignore the terminal            
                pass
            element_index += 1
    return constituents_set

In [189]:
with open("edu_segmentation/" + "file1" + ".penn") as file:
    trees_string = file.read()
    trees = trees_string.split("\n\n")

In [196]:
one_word_constituents = []
for tree in trees:
    tree_dict = {}
    for constituent in constituents(tree):
        if constituent[1] not in tree_dict:
            tree_dict[constituent[1]] = ([],[])
        tree_dict[constituent[1]][0].append(constituent)
        if constituent[2] not in tree_dict:
            tree_dict[constituent[2]] = ([],[])
        second_constituent = tree_dict[constituent[2]][1].append(constituent)
    one_word_constituents.append(tree_dict)

In [201]:
one_word_constituents[0]

{1: ([('S', 1, 78, 1), ('NP', 1, 8, 2), ('NP', 1, 2, 3), ('JJ', 1, 1, 4)],
  [('JJ', 1, 1, 4)]),
 2: ([('NN', 2, 2, 4)], [('NN', 2, 2, 4), ('NP', 1, 2, 3)]),
 3: ([('CC', 3, 3, 3)], [('CC', 3, 3, 3)]),
 4: ([('NP', 4, 8, 3), ('NNP', 4, 4, 4)], [('NNP', 4, 4, 4)]),
 5: ([('JJ', 5, 5, 4)], [('JJ', 5, 5, 4)]),
 6: ([('NN', 6, 6, 4)], [('NN', 6, 6, 4)]),
 7: ([('NN', 7, 7, 4)], [('NN', 7, 7, 4)]),
 8: ([('NNP', 8, 8, 4)], [('NNP', 8, 8, 4), ('NP', 4, 8, 3), ('NP', 1, 8, 2)]),
 9: ([('VP', 9, 77, 2), ('VBZ', 9, 9, 3)], [('VBZ', 9, 9, 3)]),
 10: ([('ADVP', 10, 10, 3), ('RB', 10, 10, 4)],
  [('ADVP', 10, 10, 3), ('RB', 10, 10, 4)]),
 11: ([('VBN', 11, 11, 4), ('VP', 11, 77, 3)], [('VBN', 11, 11, 4)]),
 12: ([('NP', 12, 24, 4), ('NNP', 12, 12, 6), ('NP', 12, 12, 5)],
  [('NNP', 12, 12, 6), ('NP', 12, 12, 5)]),
 13: ([('VBN', 13, 13, 6), ('VP', 13, 24, 5)], [('VBN', 13, 13, 6)]),
 14: ([('PP', 14, 18, 6), ('RB', 14, 14, 8), ('ADVP', 14, 14, 7)],
  [('RB', 14, 14, 8), ('ADVP', 14, 14, 7)]),
 15:

In [ ]:
# next we will compute the top syntactic tag
top_tags = []
# create a new list (it will contain dictionaries with top syntactic tags)
for tree_dict in one_word_constituents:
    tags_dict = {}
    for key, value in tree_dict.items(): # e.g. 1 --> key, ([("NP", 1, 6), ("S", 1, 44)], [("NNP", 1, 1)]) -->value
        top_tag_begin = max([(element[2], element[0]) for element in value[0]])[1]
        top_tag_end = min([(element[1], element[0]) for element in value[1]])[1]

        tags_dict.update({key: (top_tag_begin, top_tag_end)})
    top_tags.append(tags_dict)

#     Adding the Depth to the list with Top Syntactic Tags
top_depths = []
for tree_dict in one_word_constituents:
    tags_dict = {}
    for key, value in tree_dict.items():
        top_tag_begin_depth = max([(element[2], element[3]) for element in value[0]])[1]
        top_tag_end_depth = min([(element[1], element[3]) for element in value[1]])[1]

        tags_dict.update({key: (top_tag_begin_depth, top_tag_end_depth)})
    top_depths.append(tags_dict)

In [235]:
def extract_features(file_name, word2vec_model, subfolder):
    # Reading the EDU file
    RST_file = open(os.path.join("../nlp_project/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0/" + subfolder,
                                 file_name) + ".edus")
    lines = RST_file.readlines()
    RST_file.close()
    
    # tokenizing the EDUs
    tokenized_edus = []
    for edu in lines:
        tokenized_edu = []
        for token in word_tokenize(edu):
            if token == "(":
                tokenized_edu.append("OPENING_ROUND_BRACE")
            elif token == ")":
                tokenized_edu.append("CLOSING_ROUND_BRACE")
            else:
                tokenized_edu.append(token)
        tokenized_edus.append(tokenized_edu)
        
    # Remember boundary indices, combine EDUs of a sentence
    boundary_indices = [] 
    edu_boundary = []
    sentences = []
    sentence = []
    for edu in tokenized_edus:
        if (edu[-1] not in ["!", "?", "."])\
                and (edu[-2:] not in ["!\"", "?\"", ".\"", "!'", "?'", ".'"])\
                and (edu[-3:] not in ["!''", "?''", ".''"]):
            sentence.extend(edu)
            edu_boundary.append(len(sentence) - 1)
        else:
            sentence.extend(edu)
            sentences.append(sentence)
            edu_boundary.append(len(sentence) - 1)
            boundary_indices.append(edu_boundary)
            sentence = []
            edu_boundary = []
            
    # getting a POS-tag from nltk
    pos_sentences = []
    for sentence in sentences:
        pos_sentence = nltk.pos_tag(sentence)
        pos_sentences.append(pos_sentence)
        
    # writing a text in conll format
#     conll_file = open("edu_segmentation/" + file_name + ".conll", "w")
    
#     for pos_sentence in pos_sentences:
#         for word, pos in pos_sentence:
#             conll_file.write(word + " " + pos + "\n")
#         conll_file.write("\n")

#     conll_file.close()
    
    # writing the text in a file for syntactic parsing
    text_file = open("edu_segmentation/" + file_name + ".text", "w")
    text_file.write("\n".join([" ".join(sentence) for sentence in sentences]))
    text_file.close()

    os.environ["FILE_NAME"] = file_name

    # performing syntactic parsing
    os.system("./stanford-parser-full-2017-06-09/lexparser.sh edu_segmentation/\"$FILE_NAME\".text > edu_segmentation/\"$FILE_NAME\".penn")

    with open("edu_segmentation/" + file_name + ".penn") as file:
        trees_string = file.read()
        trees = trees_string.split("\n\n")
    
    one_word_constituents = []
    for tree in trees:
        tree_dict = {}
        for constituent in constituents(tree):
            if constituent[1] not in tree_dict:
                tree_dict[constituent[1]] = ([],[])
            tree_dict[constituent[1]][0].append(constituent)
            if constituent[2] not in tree_dict:
                tree_dict[constituent[2]] = ([],[])
            second_constituent = tree_dict[constituent[2]][1].append(constituent)
        one_word_constituents.append(tree_dict)
        
    # next we will compute the top syntactic tag
    top_tags = []
    # create a new list (it will contain dictionaries with top syntactic tags)
    for tree_dict in one_word_constituents:
        tags_dict = {}
        for key, value in tree_dict.items(): # e.g. 1 --> key, ([("NP", 1, 6), ("S", 1, 44)], [("NNP", 1, 1)]) -->value
            top_tag_begin = max([(element[2], element[0]) for element in value[0]])[1]
            top_tag_end = min([(element[1], element[0]) for element in value[1]])[1]

            tags_dict.update({key: (top_tag_begin, top_tag_end)})
        top_tags.append(tags_dict)
    
#     Adding the Depth to the list with Top Syntactic Tags
    top_depths = []
    for tree_dict in one_word_constituents:
        tags_dict = {}
        for key, value in tree_dict.items():
            top_tag_begin_depth = max([(element[2], element[3]) for element in value[0]])[1]
            top_tag_end_depth = min([(element[1], element[3]) for element in value[1]])[1]

            tags_dict.update({key: (top_tag_begin_depth, top_tag_end_depth)})
        top_depths.append(tags_dict)
    
    #enumerate tokens of the sentence (use *enumerate*) and iterate over them 
    # (the current token will be the middle left 4-gram element).
    data = []
    #check if there is a edu break after the token
    for (sentence_no, pos_sentence) in enumerate(pos_sentences):
        for (token_no, token) in enumerate(pos_sentence): #token_no = key
            if token in boundary_indices[sentence_no]: # taking boundary indices of a corresponding sentence
                edu_break = True
            else:
                edu_break = False

    # take a top_tag_beg from top_tags and add it to the table with sent_no
            top_tag_beg = top_tags[sentence_no][token_no +1][0] #because we've counted them starting from 1
            top_tag_end = top_tags[sentence_no][token_no +1][1]
            depth_beg = top_depths[sentence_no][token_no +1][0]
            depth_end = top_depths[sentence_no][token_no +1][1] 

    # get a vector
            word = token[0]
            POS_tag = token[1]

    # form a tuple with features which i'll put to the data

            feature_vectors = []

            feature_vectors.append(file_name)
            feature_vectors.append(sentence_no)
            feature_vectors.append(token_no)
            feature_vectors.append(word)
            feature_vectors.append(edu_break)
            feature_vectors.append(POS_tag)
            feature_vectors.append(top_tag_beg)
            feature_vectors.append(top_tag_end)
            feature_vectors.append(depth_beg)
            feature_vectors.append(depth_end)

            if word in model.vocab:
                word_vector = model.get_vector(word)
                feature_vectors.extend(list(word_vector))
            else:
                feature_vectors.extend([0]*300)

            data.append(feature_vectors)
            
    df = pd.DataFrame(data, columns=["file_name", 
                                 "sentence_no", 
                                 "token_no", 
                                 "word", 
                                 "edu_break", 
                                 "POS-tag", 
                                 "top_tag_beg", 
                                 "top_tag_end", 
                                 "depth_beg", 
                                 "depth_end"] + ["c_%.3d" % i for i in range(300)])
    
    df.to_csv("edu_segmentation/" + file_name + ".csv", index=False)
    
    return df

In [237]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [222]:
def get_file_names(ls_result, target_ext):
    file_names = []
    
    for fn in ls_result:
        base_name, ext = os.path.splitext(fn)
        if ext == target_ext:
            file_names.append(base_name)
                
    return file_names

In [13]:
def compute_features(subfolder, file_names):
    for file_name in tqdm(file_names, total=len(file_names)):
        extract_features(file_name, model, subfolder)

In [14]:
def generate_csv(subfolder):
    ls_result = os.listdir("rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0/" + subfolder)
    file_names = get_file_names(ls_result)
    compute_features(subfolder, file_names)

In [ ]:
import os
import pandas as pd

In [168]:
def add_rel_feature(vectorized_table, col_name): # a column in which I'll make changes
    vectorized_table.loc[:, "sent_max_" + col_name] = vectorized_table.groupby(["file_name", "sentence_no"]) \
                                                        [col_name].transform("max")
    vectorized_table.loc[:, "rel_" + col_name] = (vectorized_table[col_name] \
                                / vectorized_table["sent_max_" + col_name] - 0.5) * 2

In [238]:
def create_table(path, file_names, ohe=True, sent_start_no=0):    
    dfs = []
    for file in file_names:
        df = pd.read_csv(os.path.join(path, file + ".csv")) # for safety if there's a / at the end of the path
        dfs.append(df)

    df = pd.concat(dfs)
    
    if ohe:
        # transforming into vector form
        vectorized_table = pd.get_dummies(df, 
                                          columns=["POS-tag", 
                                                   "top_tag_beg", 
                                                   "top_tag_end"])
    else:
        vectorized_table = df

    sent_global_no_df = vectorized_table[["file_name", "sentence_no"]].drop_duplicates().sort_values(by="file_name")
    sent_global_no_df.loc[:, "sentence_global_no"] = range(sent_start_no,
                                                           sent_start_no + len(sent_global_no_df))
    
    vectorized_table = pd.merge(vectorized_table,
                                sent_global_no_df,
                                on=["file_name", "sentence_no"])

    vectorized_table.loc[:, "sent_word_indices"] = vectorized_table.apply(lambda row: str(row.sentence_global_no) + "_" + str(row.token_no),
                                                                          axis=1)

    vectorized_table = vectorized_table[["word"] + [c for c in vectorized_table.columns if c != "word"]]
    
    vectorized_table.loc[:, "depth_beg"] = vectorized_table.depth_beg - 1
    vectorized_table.loc[:, "depth_end"] = vectorized_table.depth_end - 1
    add_rel_feature(vectorized_table, "depth_beg")
    add_rel_feature(vectorized_table, "depth_end")
    add_rel_feature(vectorized_table, "token_no")

    vectorized_table = vectorized_table.drop(["sentence_no", 
                                              "sentence_global_no",
                                              "depth_beg",
                                              "depth_end",
                                              "token_no", 
                                              "edu_break",
                                              "sent_max_depth_beg",
                                              "sent_max_depth_end",
                                              "sent_max_token_no"], axis=1)
    vectorized_table = vectorized_table.drop([c for c in vectorized_table.columns if c.split("_", 1)[0] == "c"],
                                             axis=1)

    return vectorized_table

In [65]:
train_ls_result = os.listdir("rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0/TRAINING/")
train_file_names = get_file_names(train_ls_result, ".rst")
test_ls_result = os.listdir("rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0/TEST/")
test_file_names = get_file_names(test_ls_result, ".rst")

In [76]:
# creating a table for Training and Test files
# keeping test and training files in one table, dividing it into two tables (so that they have the same # of columns)
file_name_df = pd.DataFrame({"file_name": train_file_names + test_file_names,
                             "is_train": [True]*len(train_file_names) + [False]*len(test_file_names)})

In [239]:
vectorized_table = create_table("edu_segmentation/", 
                                train_file_names + test_file_names)

vectorized_table__split = pd.merge(vectorized_table,
                                   file_name_df,
                                   on="file_name")

train_data_set = vectorized_table__split.loc[vectorized_table__split.is_train == True] \
                                        .drop("is_train", axis=1)
test_data_set = vectorized_table__split.loc[vectorized_table__split.is_train == False] \
                                        .drop("is_train", axis=1)

train_data_set.to_csv("data_set_1__train.csv", index=False) # 1 - when test and train.files are processed together
test_data_set.to_csv("data_set_1__test.csv", index=False)

In [240]:
train_table = create_table("edu_segmentation/", train_file_names, False)

test_table = create_table("edu_segmentation/", 
                          test_file_names, 
                          False,
                          1 + max([int(sw.split("_")[0]) \
                                   for sw in train_table.sent_word_indices]))

train_table.loc[:, "is_train"] = [True] * len(train_table)
test_table.loc[:, "is_train"] = [False] * len(test_table)

table = pd.concat([train_table, test_table])

vectorized_table_2 = pd.get_dummies(table,
                                    columns=["POS-tag", 
                                             "top_tag_beg", 
                                             "top_tag_end"])

vectorized_table_2 = vectorized_table_2[["sent_word_indices",
                                         "word",
                                         "file_name"]
                                        + [c for c in vectorized_table_2.columns \
                                              if c not in ["sent_word_indices",
                                                           "word",
                                                           "file_name",
                                                           "rel_depth_beg",
                                                           "rel_depth_end",
                                                           "rel_token_no"]]
                                        + ["rel_depth_beg",
                                           "rel_depth_end",
                                           "rel_token_no"]]

train_data_set_2 = vectorized_table_2.loc[vectorized_table_2.is_train == True] \
                                        .drop("is_train", axis=1)
test_data_set_2 =  vectorized_table_2.loc[vectorized_table_2.is_train == False] \
                                        .drop("is_train", axis=1)

train_data_set_2.to_csv("data_set_2__train.csv", index=False)
test_data_set_2.to_csv("data_set_2__test.csv", index=False)

In [242]:
pd.read_csv("data_set_1__train.csv").set_index("sent_word_indices").head(39)

,word,file_name,POS-tag_#,POS-tag_$,POS-tag_'',POS-tag_(,POS-tag_),"POS-tag_,",POS-tag_.,POS-tag_:,...,top_tag_end_WHNP,top_tag_end_WHPP,top_tag_end_WP,top_tag_end_WP$,top_tag_end_WRB,top_tag_end_X,top_tag_end_``,rel_depth_beg,rel_depth_end,rel_token_no
sent_word_indices,,,,,,,,,,,,,,,,,,,,,
0_0,Energetic,file1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-1.00,-0.25,-1.000000
0_1,and,file1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.25,-0.50,-0.947368
0_2,concrete,file1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.50,-0.50,-0.894737
0_3,action,file1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.50,-0.25,-0.842105
0_4,has,file1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.25,-0.25,-0.789474
0_5,been,file1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.25,-0.25,-0.736842
0_6,taken,file1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.25,-0.25,-0.684211
0_7,in,file1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.25,-0.75,-0.631579
0_8,Colombia,file1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.75,-0.50,-0.578947


In [223]:
ls_result = os.listdir("edu_segmentation/")
text_file_names = get_file_names(ls_result, ".text")

In [225]:
file_names = train_file_names + test_file_names

In [228]:
bad_file_names = list(set(file_names).difference(set(text_file_names)))

In [ ]:
for file_name in tqdm(bad_file_names, total=len(bad_file_names)):
    if file_name in train_file_names:
        extract_features(file_name, model, "TRAINING/")
    else:
        extract_features(file_name, model, "TEST/")

  0%|          | 0/194 [00:00<?, ?it/s]

1 Exxon
2 Corp.
3 is
4 resigning
5 from
6 the
7 National
8 Wildlife
9 Federation
10 's
11 corporate
12 advisory
13 panel
14 ,
15 saying
16 the
17 conservation
18 group
19 has
20 been
21 unfairly
22 critical
23 of
24 the
25 Exxon
26 Valdez
27 oil
28 spill
29 along
30 the
31 Alaskan
32 coast
33 .
1 The
2 federation
3 said
4 Friday
5 that
6 it
7 regrets
8 the
9 resignation
10 ,

  1%|          | 2/194 [00:29<46:57, 14.67s/it]  


11 but
12 issued
13 a
14 stinging
15 response
16 that
17 called
18 Exxon
19 a
20 ``
21 corporate
22 pariah
23 ''
24 that
25 should
26 keep
27 an
28 open
29 dialogue
30 with
31 environmentalists
32 .
1 The
2 federation
3 ,
4 with
5 5.8
6 million
7 members
8 nationwide
9 ,
10 has
11 been
12 one
13 of
14 the
15 sharpest
16 critics
17 of
18 Exxon
19 's
20 handling
21 of
22 the
23 11
24 million
25 gallon
26 tanker
27 spill
28 and
29 has
30 accused
31 the
32 company
33 of
34 repeatedly
35 ignoring
36 requests
37 to
38 meet
39 and
40 discuss
41 it
42 .
1 The
2 March
3 24
4 oil
5 spill
6 soiled
7 hundreds
8 of
9 miles
10 of
11 shoreline
12 along
13 Alaska
14 's
15 southern
16 coast
17 and
18 wreaked
19 havoc
20 with
21 wildlife
22 and
23 the
24 fishing
25 industry
26 .
1 Exxon
2 's
3 Exxon
4 USA
5 unit
6 was
7 one
8 of
9 the
10 charter
11 members
12 of
13 the
14 Corporate
15 Conservation
16 Council
17 ,
18 a
19 panel
20 of
21 executives
22 formed
23 in
24 1982
25 by
26 the
27 National
28 Wild

  2%|▏         | 3/194 [00:37<39:48, 12.50s/it]

...

  5%|▍         | 9/194 [01:34<32:25, 10.52s/it]

10-a-share
43 tender
44 offer
45 for
46 the
47 company
48 .
1 The
2 offer
3 would
4 give
5 the
6 transaction
7 an
8 indicated
9 value
10 of
11 $
12 189
13 million
14 ,
15 based
16 on
17 the
18 18.9
19 million
20 shares
21 the
22 group
23 does
24 n't
25 already
26 own
27 .
1 DPC
2 holds
3 about
4 7.8
5 %
6 of
7 Dataproducts
8 '
9 shares
10 .
1 Lawyers
2 representing
3 DPC
4 declined
5 to
6 elaborate
7 ,
8 saying
9 they
10 did
11 n't
12 have
13 a
14 final
15 copy
16 of
17 the
18 filing
19 .
1 Jack
2 Davis
3 ,
4 Dataproducts
5 '
6 chairman
7 ,
8 said
9 he
10 had
11 n't
12 yet
13 seen
14 the
15 filing
16 and
17 could
18 n't
19 comment
20 .
1 DPC
2 made
3 a
4 $
5 15-a-share
6 bid
7 for
8 the
9 company
10 in
11 May
12 ,
13 but
14 Dataproducts
15 management
16 considered
17 the
18 $
19 283.7
20 million
21 proposal
22 unacceptable
23 .
1 Dataproducts
2 had
3 sought
4 a
5 buyer
6 for
7 several
8 months
9 ,
10 but
11 it
12 is
13 now
14 in
15 the
16 midst
17 of
18 a
19 restructuring
20 plan
21 an

  5%|▌         | 10/194 [01:58<36:18, 11.84s/it]


16 the
17 two
18 hotels
19 ;
20 Pan
21 American
22 ,
23 a
24 subsidiary
25 of
26 Pan
27 Am
28 Corp.
29 ,
30 will
31 have
32 a
33 10
34 %
35 share
36 .
1 The
2 Soviet
3 owners
4 will
5 be
6 Mossoviet
7 ,
8 Moscow
9 's
10 city
11 governing
12 body
13 ,
14 and
15 Aeroflot
16 ,
17 the
18 Soviet
19 national
20 airline
21 .
1 Although
2 a
3 Finnish
4 group
5 has
6 a
7 minority
8 interest
9 in
10 an
11 already
12 operating
13 Moscow
14 hotel
15 ,
16 the
17 Sheraton-Pan
18 Am
19 venture
20 will
21 be
22 the
23 first
24 joint-venture
25 hotels
26 in
27 the
28 Soviet
29 Union
30 to
31 have
32 as
33 much
34 as
35 50
36 %
37 foreign
38 ownership
39 .
1 U.S.
2 companies
3 account
4 for
5 less
6 than
7 8
8 %
9 of
10 the
11 1,000
12 or
13 more
14 Soviet
15 joint
16 ventures
17 that
18 have
19 been
20 announced
21 since
22 the
23 Soviets
24 began
25 encouraging
26 such
27 undertakings
28 in
29 1987
30 .
1 But
2 some
3 U.S.
4 companies
5 are
6 negotiating
7 projects
8 that
9 could
10 be
11 among
12 th

  6%|▌         | 11/194 [02:23<39:55, 13.09s/it]